# 第15章 生存分析


## 15.1 生存データ

### データの読み込み

In [ ]:
import polars as pl

# id: id
# rx: 治療(1 = プラシーボ, 2 = チオテパ)
# number: 最初の腫瘍の数(8は8以上)
# size: 最初の腫瘍の最大サイズ[cm]
# stop: 再発または打ち切りが、いつ発生したか
# event: 膀胱癌の再発(0: No, 1: Yes)
# enum: 再発が何回目か(4まで)
bladder = pl.read_csv("../data/bladder.csv")
display(bladder.head())

display(bladder["rx"].value_counts())

## 15.2 カプランマイヤー曲線

In [ ]:
from lifelines import KaplanMeierFitter

# 学習
kmf = KaplanMeierFitter()
kmf.fit(bladder["stop"], event_observed = bladder["event"])

# 生存曲線を描画する
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
kmf.plot(ax = ax)
ax.set_title("Survival with confidence intervals")
plt.show()

## 15.3 Coxの比例ハザードモデル

In [ ]:
from lifelines import CoxPHFitter

# 学習に使用する列を抽出する
df_study = bladder[["rx", "number", "size", "enum", "stop", "event"]]

# 学習する
cph = CoxPHFitter()
cph.fit(df_study.to_pandas(), duration_col = "stop", event_col = "event")

# 学習結果を表示する
cph.print_summary()


## 15.3 Coxの比例ハザードモデル

In [ ]:
# 1 = プラシーボ
# 2 = チオテパ
rx1 = bladder.filter( pl.col("rx") == 1 )
rx2 = bladder.filter( pl.col("rx") == 2 )

# 層別に生存曲線を学習する
kmf1 = KaplanMeierFitter()
kmf1.fit(rx1["stop"], event_observed = rx1["event"])

kmf2 = KaplanMeierFitter()
kmf2.fit(rx2["stop"], event_observed = rx2["event"])

# 生存曲線を描画する
fig, axes = plt.subplots()

kmf1.plot_loglogs(ax = axes)
kmf2.plot_loglogs(ax = axes)

axes.legend(["rx1", "rx2"])

plt.show()

In [ ]:
cph_strat = CoxPHFitter()
cph_strat.fit(
    df_study.to_pandas(),
    duration_col = "stop",
    event_col = "event",
    strata = ["rx"]
)
cph_strat.print_summary()